In [1]:
from openpile.construct import Pile, SoilProfile, Layer, Model
from openpile.soilmodels import API_clay, API_sand, frankeRollins2013
from openpile.analyze import winkler
from scipy.optimize import fsolve

import pandas as pd
import os

In [2]:
# Get the current working directory
cwd = os.getcwd()
# Read the CSV file (replace 'filename.csv' with your actual file name)
dfPileConfig = pd.read_csv(os.path.join(cwd, 'pileConfig-pier1-seismic.csv'))
dfPileConfig.index.name = 'id'
dfPileConfig

,diameter[m],length[m],factored-axial-load[kN],initial-lateral-load[kN]
id,,,,
0,1.5,30,6995,255
1,1.5,35,8990,245
2,1.5,40,10975,245
3,1.5,45,12960,240
4,1.5,50,14940,225
5,2.0,30,9990,715
6,2.0,35,12670,695
7,2.0,40,15340,700
8,2.0,45,18000,700


In [3]:
sp = SoilProfile(
    name="pier1-seismic",
    top_elevation=0.0,
    water_line=14.80,
    layers=[
        Layer(
            name="L. to M.Dense Clean Sand [LIQ] S_res=28.6 kPa, e50=0.56%, γsat=17kN/m³",
            top=0,
            bottom=-3.0,
            weight=17.0,
            lateral_model=frankeRollins2013(Sr=28.64070281, eps50=0.005612657, kind="static"),
        ),
        Layer(
            name="L. to M.Dense Clean Sand [LIQ] Φ=31°, S_res=36.7 kPa, e50=0.055%, γsat=18kN/m³",
            top=-3.0,
            bottom=-5.0,
            weight=18.0,
            lateral_model=frankeRollins2013(Sr=36.73006161, eps50=0.000554117, kind="static"),
        ),
        Layer(
            name="L. to M.Dense Clean Sand Φ=31°, γsat=18kN/m³",
            top=-5.0,
            bottom=-6.0,
            weight=18.0,
            lateral_model=API_sand(phi=31, kind="static", georgiadis=True, d_adj=-2.7503495720876026),
        ),
        Layer(
            name="M.Dense Silty Sand Φ=35°, γsat=18kN/m³",
            top=-6.0,
            bottom=-12.0,
            weight=18.0,
            lateral_model=API_sand(phi=35, kind="static", georgiadis=True, d_adj=-3.430764357553343),
        ),
        Layer(
            name="M. to Dense Clean Sand Φ=36°, γsat=18kN/m³",
            top=-12.0,
            bottom=-18.0,
            weight=18.0,
            lateral_model=API_sand(phi=36, kind="static", georgiadis=True, d_adj=-3.918166559763933),
        ),
        Layer(
            name="Dense Silty Sand Φ=37°, γsat=19kN/m³",
            top=-18.0,
            bottom=-24.0,
            weight=19.0,
            lateral_model=API_sand(phi=37, kind="static", georgiadis=True, d_adj=-4.710384583222854),
        ),
        Layer(
            name="Hard Clay Su=192kPa, e50=0.32%, γsat=20kN/m³",
            top=-24.0,
            bottom=-30.0,
            weight=20.0,
            lateral_model=API_clay(Su=192, eps50=0.32/100, kind="static"),
        ),
        Layer(
            name="D. to V.Dense Sand Φ=39°, γsat=19kN/m³",
            top=-30.0,
            bottom=-60.0,
            weight=19.0,
            lateral_model=API_sand(phi=39, kind="static", georgiadis=True, d_adj=-14.37335596671765),
        ),
    ],
)

In [4]:
from openpile.utils.py_curves import frankeRollins2013

y, p =    frankeRollins2013(sig=29.0,
    X=4,
    Sr=36.73006161,
    eps50=0.000554117,
    D=2.0,
    ymax=0.05)



In [5]:
df_p = pd.DataFrame(p)
df_p.to_clipboard(index=False, header=False) 

In [6]:
df_y = pd.DataFrame(y)
df_y.to_clipboard(index=False, header=False) 

In [7]:
deflectionList = []
forceList = []
pySpringList = []
resultList = []

# List solved shear values
solutions = []

for index, row in dfPileConfig.iterrows():
    print(f"""
          
          ========================================================================
          diameter: {row['diameter[m]']} m\t length: {row['length[m]']} m
          ========================================================================

          """)

    p = Pile(name = "pier1-seismic",
        kind='Solid-Circle',
        material='Concrete',
        top_elevation = 14.80,
        pile_sections={
            'length': [row['length[m]'] + 14.80],
            'diameter': [row['diameter[m]']],
            'wall thickness': [1e-10],
            'E': [248700620.3], # Young's Modulus
            # 'area': [1.767146],
            # 'I': [2.48505],
            # 'width': [1.5],
            # 'tip_area': [ 1.767146]
            
        }
    )

    # Create Model
    M = Model(name="pier1-seismic", 
            pile=p, 
            soil=sp, 
            element_type="Timoshenko",
            coarseness=0.2)
    
    # Apply bottom fixity along x-axis
    M.set_support(elevation= (-1 * row['length[m]']), Tx=True)
    M.set_support(elevation=14.80, Rz=True)

    axial = row['factored-axial-load[kN]']

    M.set_pointload(elevation=14.80, Px=-axial)
    M.set_pointdisplacement(elevation=14.80, Ty=0.049999)

    Result = winkler(M)

    topDeflection = Result.deflection['Deflection [m]'].iloc[0]

    topShear = Result.forces[["V [kN]"]].iloc[0]

    print(f"\nSolved shear value corresponding to {topDeflection} mm deflection: {topShear[0]:.4f} kN\n")

    solutions.append(topShear.iloc[0])

    print(f"""
    id: {index}\tdiameter: {row['diameter[m]']} m\tlength: {row['length[m]']} m\tAxial: {row['factored-axial-load[kN]']} kN\tShear: {topShear.iloc[0]:.4f} kN\tDeflection: {float(Result.deflection['Deflection [m]'].iloc[0])*1000:.4f} mm
    """)
    
    deflectionList.append(Result.deflection)  # Append the solution to the list
    forceList.append(Result.forces)
    pySpringList.append(M.get_py_springs(kind='element'))
    resultList.append(Result)

    # # Solve for shear load
    # def analysis(axial, shear):

    #     M.set_pointload(elevation=14.80, Px=axial, Py=shear)
    #     # Run analysis

    #     Result = winkler(M)

    #     print(f"Axial: {axial} kN\tShear: {float(shear):.4f} kN\tDeflection: {float(Result.deflection['Deflection [m]'].iloc[0])*1000:.4f} mm")

    #     return Result.deflection['Deflection [m]'].iloc[0] 

    # goal = 0.024999  # allowable deflection of 0.025 m

    # solution = fsolve(lambda x: analysis(axial = row['factored-axial-load[kN]'], shear = x) - goal, x0=row['initial-lateral-load[kN]'],xtol=1e-3, maxfev=4)

    # print(f"\nSolved shear value corresponding to {goal*1000} mm deflection: {solution[0]:.4f} kN\n")

    # solutions.append(solution[0])  # Append the solution to the list

# Add solutions as a new column to the DataFrame
dfPileConfig['solved-lateral-load[kN]'] = solutions




          
          diameter: 1.5 m	 length: 30.0 m

          
Converged at iteration no. 2

Solved shear value corresponding to 0.049999 mm deflection: 249.2521 kN


    id: 0	diameter: 1.5 m	length: 30.0 m	Axial: 6995.0 kN	Shear: 249.2521 kN	Deflection: 49.9990 mm
    

          
          diameter: 1.5 m	 length: 35.0 m

          


c:\Users\AMH-L143\anaconda3\envs\openPile-mk2\lib\site-packages\pandas\core\frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


Converged at iteration no. 2

Solved shear value corresponding to 0.049999 mm deflection: 244.4197 kN


    id: 1	diameter: 1.5 m	length: 35.0 m	Axial: 8990.0 kN	Shear: 244.4197 kN	Deflection: 49.9990 mm
    

          
          diameter: 1.5 m	 length: 40.0 m

          
Converged at iteration no. 2

Solved shear value corresponding to 0.049999 mm deflection: 239.6069 kN


    id: 2	diameter: 1.5 m	length: 40.0 m	Axial: 10975.0 kN	Shear: 239.6069 kN	Deflection: 49.9990 mm
    

          
          diameter: 1.5 m	 length: 45.0 m

          
Converged at iteration no. 2

Solved shear value corresponding to 0.049999 mm deflection: 234.7895 kN


    id: 3	diameter: 1.5 m	length: 45.0 m	Axial: 12960.0 kN	Shear: 234.7895 kN	Deflection: 49.9990 mm
    

          
          diameter: 1.5 m	 length: 50.0 m

          
Converged at iteration no. 2

Solved shear value corresponding to 0.049999 mm deflection: 229.9790 kN


    id: 4	diameter: 1.5 m	length: 50.0 m	Axial: 14940.0 kN	Shear: 229

In [10]:
import pickle

with open('pier1-seismic/dfPileConfig.pkl', 'wb') as file:
    pickle.dump(dfPileConfig, file)
with open('pier1-seismic/deflectionList.pkl', 'wb') as file:
    pickle.dump(deflectionList, file)
with open('pier1-seismic/forceList.pkl', 'wb') as file:
    pickle.dump(forceList, file)
with open('pier1-seismic/pySpringList.pkl', 'wb') as file:
    pickle.dump(pySpringList, file)
with open('pier1-seismic/modelList.pkl', 'wb') as file:
    pickle.dump(modelList, file)
with open('pier1-seismic/resultList.pkl', 'wb') as file:
    pickle.dump(resultList, file)

MemoryError: 

In [1]:
import pickle

# Load from file
with open('pier1-seismic/dfPileConfig.pkl', 'rb') as file:
    dfPileConfig = pickle.load(file)
with open('pier1-seismic/deflectionList.pkl', 'rb') as file:
    deflectionList = pickle.load(file)
with open('pier1-seismic/forceList.pkl', 'rb') as file:
    forceList = pickle.load(file)
with open('pier1-seismic/pySpringList.pkl', 'rb') as file:
    pySpringList = pickle.load(file)
with open('pier1-seismic/modelList.pkl', 'rb') as file:
    modelList = pickle.load(file)
with open('pier1-seismic/resultList.pkl', 'rb') as file:
    resultList = pickle.load(file)

In [9]:
# dfPileConfig.to_clipboard(index=True,sep=",")
forceList[9].to_clipboard(index=False,sep=",")
# deflectionList[9].to_clipboard(index=False,sep=",")
# pySpringList[9].to_clipboard(index=False,sep=",")
# modelList[9].plot()

In [21]:
filtered_df = pySpringList[9][(pySpringList[9]['Elevation [m]'] == -50)]
filtered_df = filtered_df[filtered_df['Position'] == 'bottom']
filtered_df.to_clipboard(index=False,sep=",")

In [ ]:
filtered_df

In [10]:
deflectionList[9].to_clipboard(index=True)

In [11]:
pySpringList[9].to_clipboard(index=True)

In [ ]:
modelList[9].plot()

In [24]:
resultList[9].py_mobilization.to_clipboard(index=False,sep=",")

In [ ]:
df = modelList[9].get_soil_properties()
df.loc[df['x_top [m]'] == -2.00]

In [ ]:
import matplotlib.pyplot as plt
from openpile.utils.py_curves import api_sand, frankeRollins2013

p, y = frankeRollins2013(
    sig=14, 
    X=2,  
    Sr=28.64070281,
    eps50=0.005612657,
    D=2.0
)

plt.plot(p, y)
plt.ylabel("p [kN/m]")
plt.xlabel("y [m]")


In [ ]:
p